# electricity_market_env

> Fill in a module description here

In [ ]:
#| default_exp electricity_market_env

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import gymnasium as gym
import numpy as np
from gymnasium.envs.registration import register

In [ ]:
#| export
class ElectricityMarketEnv(gym.Env):
    def __init__(self, env_config=None):
        # Environment Configuration
        if env_config is None:
            env_config = {}
        # State of Environment
        self._timestep = 0
        self._init_state_of_charge = env_config.get('init_state_of_charge', 50)
        self._current_state_of_charge = env_config.get('init_state_of_charge', 100)
        self._battery_capacity = env_config.get('battery_capacity', 100)
        self._battery_efficiency = 0.95
        self._battery_degradation = 0.999
        self._config = env_config

        self.action_space = gym.spaces.Discrete(2 * self._battery_capacity + 1)
        self.actions = list(range(-self._battery_capacity, self._battery_capacity + 1))
        # observation space: (Battery SoC, Current electricity demand, Current market price)
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0]),
                                                high=np.array([self._battery_capacity, np.inf, np.inf]),
                                                shape=(3,), dtype=np.float64)


    def _is_action_valid(self, action) -> bool:
        charge_amount = self.actions[action]
        target_state_of_charge = self._current_state_of_charge + charge_amount * self._battery_efficiency
        return 0 < target_state_of_charge < self._battery_capacity

    def step(self, action):
        charge_amount = self.actions[action]
        self._timestep += 1
        done = self._battery_efficiency == 0
        truncated = False

        demand = self._demand_of_electricity()
        price = self._price()
        if not self._is_action_valid(action):
            truncated = True
            return self._get_obs(), -1000, done, truncated, {}

        self._current_state_of_charge += charge_amount * self._battery_efficiency

        self._battery_capacity *= self._battery_degradation
        reward = self._calculate_reward(charge_amount, demand, price)
        return self._get_obs(), reward, done, truncated, {}

    def _calculate_reward(self, charge_amount, demand, price):
        """ Reward function considering price, demand, battery health, and grid stability. """
        if charge_amount > 0:
            reward = -demand * price
        else:
            reward = (charge_amount - demand) * price
        if self._current_state_of_charge < 20 or self._current_state_of_charge > 80:
            reward *= 0.9

        return reward

    def _demand_of_electricity(self):
        base_demand = 100 + 50 * np.sin(self._timestep * 2 * np.pi)  # Cyclical demand
        noise = np.random.normal(0, 10)  # Random noise
        return max(0, base_demand + noise)

    def _price(self):
        """ Simulates electricity price changes based on demand and supply. """
        base_price = 30 + 50 * np.sin(self._timestep * 2 * np.pi)  # Cyclical price changes
        noise = np.random.normal(0, 5)  # Random price fluctuations
        return max(0, base_price + noise)

    def reset(self, *, seed=None, options=None):
        """ Resets the environment to the initial state. """
        super().reset(seed=seed)
        self._timestep = 0
        self._init_state_of_charge = self._config.get('init_state_of_charge', 50)
        self._current_state_of_charge = self._config.get('init_state_of_charge', 100)
        self._battery_capacity = self._config.get('battery_capacity', 100)
        self._battery_efficiency = 0.95
        self._battery_degradation = 0.999
        return self._get_obs(), {}

    def action_masks(self):
        """Generate a boolean mask of valid actions for `MaskablePPO`."""
        return np.array([self._is_action_valid(i) for i in range(self.action_space.n)], dtype=bool)

    def _get_obs(self):
        """ Returns the current observation (state). """
        return np.array([
            self._current_state_of_charge,  # Battery SoC
            self._demand_of_electricity(),  # Current electricity demand
            self._price()  # Current market price
        ])



In [ ]:
#| export
register(
    id='ElectricityMarketEnv-v0',
    entry_point='electricity_market.electricity_market_env:ElectricityMarketEnv',
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()